In [2]:
import os
import pandas as pd
import geopandas as gpd
from utils import *
import zipfile

In [29]:
charged_veh_zip = '../inputs/charged_vehicles/012_Tolled_Veh_Data.zip'
charged_veh_dfs = []

with zipfile.ZipFile(charged_veh_zip , 'r') as z:
    for f_dir in z.namelist():
        if f_dir.endswith('.csv'):
            with z.open(f_dir) as f:
                temp_df = pd.read_csv(f, skiprows=9, index_col=0)[:3].reset_index()
                temp_col = temp_df.columns.tolist()[1:] + ['-']
                temp_df.columns = temp_col
                temp_df = temp_df.drop(columns=['-'])

                charged_veh_dfs.append(temp_df)

charged_veh_df = pd.concat(charged_veh_dfs)
charged_veh_df['slot_no'] = charged_veh_df['Scenario_No'].astype(str).str[-2:].astype(int)

charged_veh_df.to_csv('../outputs/charged_vehs/charged_vehs_p2-1.csv', index=None)

In [13]:
def mx_to_ijk(df, metrics, slot_no):
    df = pd.melt(df, id_vars = ['origin'], 
                value_vars = df.columns.tolist().remove('origin'), 
                var_name = 'destination', 
                value_name = f'{metrics}')
    df['slot_no'] = slot_no
    df['scenario'] = REVERSE_SCENARIO_MAP[slot_no]
    return df

In [15]:
charged_veh_zip = '../inputs/charged_vehicles/Number_of_vehicles_charged_PM.zip'
charged_veh_dfs = []

with zipfile.ZipFile(charged_veh_zip , 'r') as z:
    for f_dir in z.namelist():
        if f_dir.endswith('.csv'):
            for scenario in SCENARIO_MAP:
                slot_no = SCENARIO_MAP[scenario]
                model_code = f"263{str(slot_no)}"
                if model_code in f_dir:
                    with z.open(f_dir) as f:
                        temp_df = pd.read_csv(f, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                        temp_df = mx_to_ijk(temp_df, "Charged_Veh_PM", slot_no)
                        temp_df['origin'] = temp_df['origin'].astype(int)
                        temp_df['destination'] = temp_df['destination'].astype(int)
                        charged_veh_dfs.append(temp_df)

charged_veh_df = pd.concat(charged_veh_dfs)

In [17]:
charged_veh_df.to_csv('../outputs/charged_vehs/chargedveh_pm_od.csv', index=None)
charged_veh_df.groupby(['origin', 'slot_no','scenario']
                       ).agg({'Charged_Veh_PM':'sum'}
                             ).reset_index().to_csv('../outputs/charged_vehs/chargedveh_pm_o.csv', index=None)
charged_veh_df.groupby(['destination', 'slot_no','scenario']
                       ).agg({'Charged_Veh_PM':'sum'}
                             ).reset_index().to_csv('../outputs/charged_vehs/chargedveh_pm_d.csv', index=None)

WITH TOLL BANDS

In [9]:
charged_veh_zip = '../inputs/charged_vehicles/015_Tolled_Band_Veh_Skim_NG.zip'
charged_veh_band_dfs = []

with zipfile.ZipFile(charged_veh_zip , 'r') as z:
    for f_dir in z.namelist():
        if f_dir.endswith('.csv'):
            with z.open(f_dir) as f:
                temp_df = pd.read_csv(f, skiprows=10, index_col=0)[:42].reset_index()
                temp_col = temp_df.columns.tolist()[1:] + ['-']
                temp_df.columns = temp_col
                temp_df = temp_df.drop(columns=['-'])

                charged_veh_band_dfs.append(temp_df)

charged_veh_band_df = pd.concat(charged_veh_band_dfs)
charged_veh_band_df['slot_no'] = charged_veh_band_df['Scenario_No'].astype(str).str[-2:].astype(int)

charged_veh_band_df.to_csv('../outputs/charged_vehs/charged_vehs_band_p2-1.csv', index=None)